In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F

B: batch dimension  
T: sequence length  
D: feature dimension (hidden_size)  

In [21]:
torch.manual_seed(1337)
# B: batch dimension, T: sequence length, D: feature dimension (hidden size)
B,T,D = 4,8,2
x = torch.randn(B,T,D)
xbow = torch.zeros((B,T,D))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1,:] #(T,D)
        xbow[b,t,:] = torch.mean(xprev,dim=0) # reduce the time dimension

x.shape == xbow.shape

True

We average the previous and current tokens.  
![image](images/bow.png)

In [22]:
print(f'The first batch of x is \n{x[0]}')
print(f'The first batch of xbow is \n{xbow[0]}')

The first batch of x is 
tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])
The first batch of xbow is 
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])


Use matrix multiplication to calculate the mean of the previous tokens.

In [23]:
a = torch.tril(torch.ones(3,3)) # lower traingular matrix
a = a / torch.sum(a,dim=1,keepdim=True) # sum up the rows
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print(f'a=\n{a}')
print(f'b=\n{b}')
print(f'c=\n{c}')


a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b=
tensor([[8., 6.],
        [5., 2.],
        [4., 4.]])
c=
tensor([[8.0000, 6.0000],
        [6.5000, 4.0000],
        [5.6667, 4.0000]])


In [24]:
# a stands for attention matrix
# b stands for input matrix
# c stands for output matrix
attn = torch.tril(torch.ones(T,T))
attn = attn / torch.sum(attn,dim=1,keepdim=True) # sum up the rows
print(f'attn matrix: \n{attn}')
xbow2 = attn @ x #(T,T) @ (B,T,D) -> (B,T,D)
print(f'The first batch of xbow2 is \n{xbow2[0]}')

attn matrix: 
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])
The first batch of xbow2 is 
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])


We can also use softmax to produce the attention weights.

In [25]:
tril = torch.tril(torch.ones(T,T))
attn = torch.zeros(T,T)
attn = attn.masked_fill(tril == 0, float('-inf')) # mask out the upper triangle (masked-attention)
attn = F.softmax(attn,dim=-1) # reduce the time dimension
xbow3 = attn @ x
print(f'attn matrix: \n{attn}')
print(f'The first batch of xbow3 is \n{xbow3[0]}')

attn matrix: 
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])
The first batch of xbow3 is 
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])


### Self attention

Previous attention matrix simply averages the previous tokens and current tokens.  
Can we modify the weight matrix to pay more attention to specific token?  
Yes! We can use self-attention machanism to do so.  

In [26]:
B,T,D = 1,8,32
x = torch.randn(B,T,D)

head_size = 16
key = nn.Linear(D, head_size)
query = nn.Linear(D, head_size)

k = key(x) # (B,T,D) -> (B,T,head_size)
q = query(x) # (B,T,D) -> (B,T,head_size)

# previous, we simply set weight matrix to be all zeros
attn = k @ q.transpose(2,1) # (B,T,head_size) @ (B,head_size,T) -> (B,T,T)
tril = torch.tril(torch.ones(T,T))
attn = attn.masked_fill(tril == 0, float('-inf')) # mask out the upper triangle (masked-attention)
attn = F.softmax(attn,dim=-1)
print(f'attention matrix: \n{attn}')

attention matrix: 
tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.8568, 0.1432, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.9035, 0.0319, 0.0646, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0794, 0.7826, 0.0262, 0.1117, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2599, 0.0619, 0.0666, 0.5537, 0.0579, 0.0000, 0.0000, 0.0000],
         [0.1648, 0.0910, 0.0811, 0.1171, 0.1276, 0.4185, 0.0000, 0.0000],
         [0.5038, 0.0824, 0.0079, 0.2029, 0.0508, 0.0840, 0.0683, 0.0000],
         [0.1233, 0.1467, 0.1079, 0.1274, 0.0938, 0.3208, 0.0458, 0.0343]]],
       grad_fn=<SoftmaxBackward0>)


Previously, we do dot product between attention weights and the embedding of the tokens. 
Here, we perform dot product between the attention weights and "values" of the tokens.  The value is produced by passing the embedding through a linear layer.

In [27]:
value = nn.Linear(D,head_size)
v = value(x) # (B,T,D) -> (B,T,head_size)
o = attn @ v # (B,T,T) @ (B,T,head_size) -> (B,T,head_size)
o.shape # (B,T,head_size)

torch.Size([1, 8, 16])

We pass the dot product through a linear layer called output layer.

In [28]:
out = nn.Linear(head_size,D)
o = out(o) # (B,T,head_size) -> (B,T,D)
o.shape # (B,T,head_size)
print(f'the input dimension is {x.shape} and the output dimension is {o.shape}')

the input dimension is torch.Size([1, 8, 32]) and the output dimension is torch.Size([1, 8, 32])


### Scaled product attention
We expect the variance to be close to 1, but after dot product, the variance will be close to head_size. We therefore divide the dot product by sqrt(head_size) to scale the variance back to 1.

In [29]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
print('The variance of k is', torch.var(k).item())
print('The variance of q is', torch.var(q).item())
attn = k @ q.transpose(2,1)
print('The variance of attn is', torch.var(attn).item())


The variance of k is 1.1766988039016724
The variance of q is 0.8625323176383972
The variance of attn is 16.994678497314453


In [30]:
attn = k @ q.transpose(2,1) / (head_size**0.5)
print('The variance of attn is', torch.var(attn).item())

The variance of attn is 1.0621674060821533


In [40]:
class SHATTN(nn.Module):
    '''
    A simple single head of the self-attention mechanism.
    '''
    def __init__(self, hidden_size, head_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.head_size = head_size
        self.query = nn.Linear(hidden_size, head_size)
        self.key = nn.Linear(hidden_size, head_size)
        self.value = nn.Linear(hidden_size, head_size)
        self.out = nn.Linear(head_size, hidden_size)
    def forward(self, x):
        B,T,D = x.shape
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)
        # compute attention matrix
        attn = (q @ k.transpose(2, 1)) / (self.head_size ** 0.5)
        attn = attn.masked_fill(torch.ones(T,T,device=x.device,dtype=torch.bool) == 0, float('-inf'))
        attn = torch.softmax(attn, dim=-1) 
        o = attn @ v
        out = self.out(o)
        return out
    
x = torch.randn(B,T,D)
attn = SHATTN(D,head_size)
o = attn(x)
print(f'the input dimension is {x.shape} and the output dimension is {o.shape}')


the input dimension is torch.Size([1, 8, 32]) and the output dimension is torch.Size([1, 8, 32])


In [41]:
class MHATTN(nn.Module):
    '''
    Implementation of multi-head attention mechanism.
    '''
    def __init__(self, hidden_size,num_heads):
        super().__init__()
        self.num_heads = num_heads
        self.head_size = hidden_size // num_heads
        self.heads = nn.ModuleList([SHATTN(self.head_size, self.head_size) for _ in range(num_heads)])
        self.out = nn.Linear(hidden_size, hidden_size)
    def forward(self, x):
        # slice the hidden_size dimension into num_heads
        B,T,D = x.shape
        x = x.view(B,T,-1,self.head_size).transpose(1, 2)
        # concatenate the heads along the hidden_size dimension
        o = torch.cat([self.heads[i](x[:,i,:,:]) for i in range(self.num_heads)], dim=-1)
        out = self.out(o)
        return out
 
x = torch.randn(B,T,D)
attn = MHATTN(D,num_heads=4)
o = attn(x)
print(f'the input dimension is {x.shape} and the output dimension is {o.shape}')

the input dimension is torch.Size([1, 8, 32]) and the output dimension is torch.Size([1, 8, 32])
